# Introduction

### Description

Many machine learning algorithms can not work with categorical data. One approach to go over this obstacle is to use one-hot encoding. However, if categorical feature takes `k` distinct values and there are `n` objects in a dataset, `n` * `k` values will be stored after dense one-hot encoding. In case of high `k`, this can cause `MemoryError`. Of course, there are several approaches that do not result in extreme growth of learning sample size even if categorical features have high cardinality. For example, hashing trick is implemented in `sklearn.feature_extraction.FeatureHasher`. In this demo, another memory-efficient approach is shown.

Suppose that we want to replace categorical feature with mean of target variable conditional on this feature (i.e., group by the feature, average target within each group, and replace each value of the feature with average of its group). Is it a good idea? At the first glance, yes, it is. Nevertheless, the devil is in the detail. The smaller a group of objects with a particular value of the feature is, the higher contribution of each object's target to within-group average is. Random noise from target variable leaks to a new feature and it is not milded by averaging over a sample if this sample is not big enough. Hence, this new feature gives an advantage that is unfair, because this advantage is present during training, but it is absent when new examples with unknown target are provided.

A cure to this problem is to compute aggregates of target with usage of other object's targets, but without target of an object for which a new feature is being computed right now. For example, new feature can be generated out-of-fold: data are split into folds and a new feature for a fold is computed on all other folds only. It is like stacking where a first-stage model does not use any features directly and just applies an aggregating function to a target with respect to grouping by a categorical feature under processing.

Classes `OutOfFoldTargetEncodingRegressor` and `OutOfFoldTargetEncodingClassifier` from `dsawl.target_encoding.estimators` are plug-and-play implementations of this trick. They have `sklearn`-compatible API such that cross validation scores measured by means of `sklearn` are realistic.

### References

Look at [this Kaggle post](https://www.kaggle.com/general/16927#95887) for additional details.

### General Preparations

In [1]:
from typing import List, Tuple

import numpy as np
import pandas as pd

from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

from dsawl.target_encoding.estimators import OutOfFoldTargetEncodingRegressor

In [2]:
np.random.seed(361)

# Synthetic Dataset Generation

In [3]:
def generate_data(
        slopes: List[float],
        group_size: int,
        noise_stddev: float
        ) -> pd.DataFrame:
    """
    Generate `len(slopes)` * `group_size` examples
    with dependency y = slope * x + noise.
    """
    dfs = []
    for i, slope in enumerate(slopes):
        curr_df = pd.DataFrame(columns=['x', 'category', 'y'])
        curr_df['x'] = range(group_size)
        curr_df['category'] = i
        curr_df['y'] = curr_df['x'].apply(
            lambda x: slope * x + np.random.normal(scale=noise_stddev)
        )
        dfs.append(curr_df)
    df = pd.concat(dfs)
    return df

Let us create a situation where in-fold generation of target-based features leads to overfitting. To do so, make a lot of small categories and set noise variance to a high value. Such settings result in leakage of noise from target into in-fold generated mean. Thus, regressor learns how to use this leakage, which is useless on hold-out sets.

In [4]:
slopes = [2, 1, 3, 4, -1, -2, 3, 2, 1, 5, -2, -3, -5, 8, 1, -7, 0, 2, 0]
group_size = 5
noise_stddev = 10

In [5]:
train_df = generate_data(slopes, group_size, noise_stddev)
train_df.head()

,x,category,y
0,0,0,3.314436
1,1,0,-4.654332
2,2,0,1.642263
3,3,0,15.854888
4,4,0,21.873901


Generate test set from the same distribution.

In [6]:
test_df = generate_data(slopes, group_size, noise_stddev)
test_df.head()

,x,category,y
0,0,0,-0.682203
1,1,0,9.033670
2,2,0,8.911399
3,3,0,2.345571
4,4,0,3.408391


# Benchmark: Training with In-Fold Generated Mean

In [7]:
encoding_df = train_df \
    .groupby('category', as_index=False) \
    .agg({'y': np.mean}) \
    .rename(columns={'y': 'infold_mean'})

In [8]:
def get_target_and_features(df: pd.DataFrame) -> Tuple[pd.DataFrame]:
    merged_df = df.merge(encoding_df, on='category')
    X = merged_df[['x', 'infold_mean']]
    y = merged_df['y']
    return X, y

In [9]:
X_train, y_train = get_target_and_features(train_df)
X_test, y_test = get_target_and_features(test_df)

In [10]:
rgr = LinearRegression()

In [11]:
rgr.fit(X_train, y_train)
y_hat_train = rgr.predict(X_train)
r2_score(y_train, y_hat_train)

0.42987610029007461

In [12]:
y_hat = rgr.predict(X_test)
r2_score(y_test, y_hat)

0.061377490670754042

Obviously, overfitting is detected.

# Out-of-Fold Target Encoding Estimator

In [13]:
# As of now, `dsawl.target_encoding` does not support `pandas`, so add `.values` after dataframes.
X_train, y_train = train_df[['x', 'category']].values, train_df['y'].values
X_test, y_test = test_df[['x', 'category']].values, test_df['y'].values

In [14]:
splitter = KFold(shuffle=True, random_state=361)
rgr = OutOfFoldTargetEncodingRegressor(
    LinearRegression(),  # No arguments should be passed to constructor here.
    dict(),  # If neeeded, pass constructor arguments here as a dictionary.
    # Separation of constructor arguments from estimator makes code
    # with involvement of tools such as `GridSearchCV` more consistent.
    splitter=splitter,  # Define how to make folds for features generation.
    smoothing_strength=0,  # New feature can be smoothed towards unconditional aggregate.
    min_frequency=1,  # Unconditional aggregate is used for rare enough values.
    drop_source_features=True  # To use or not to use features from conditioning.
)

What is below is a wrong way to measure train error. Regressor uses in-fold generated features for predictions, not the features that are used for training.

In [15]:
rgr.fit(X_train, y_train, source_positions=[1])
y_hat_train = rgr.predict(X_train)
r2_score(y_train, y_hat_train)

0.32858825879553122

Now, let us look at the right way to measure performance on train set. In `OutOfFoldFeaturesRegressor` and `OutOfFoldFeaturesClassifier` `fit_predict` method is not just a combination of `fit` and `predict` methods.

In [16]:
y_hat_train = rgr.fit_predict(X_train, y_train, source_positions=[1])
r2_score(y_train, y_hat_train)

0.13534718308949856

In [17]:
rgr.fit(X_train, y_train, source_positions=[1])
y_hat = rgr.predict(X_test)
r2_score(y_test, y_hat)

0.13394140594498127

Thus, there is no blatant overfitting, because train set score and test set score are close to each other (for some other random seeds the gap between them might be higher, but, anyway, it is not too considerable). Also it is appropriate to highlight that test set score is significantly higher than that of the benchmark regressor trained with in-fold generated mean.